In [3]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [4]:
dataset = load_dataset("chirunder/text_messages")

In [5]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df.head()

,texto
0,Top right I gained a little speed with the add...
1,They are heavier wheels though as are all the ...
2,Federally registering a trademark is more than...
3,I'll have to jog my memory from rooting a few ...
4,Unless you can afford to buy all new larger cl...


Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [6]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [18]:
df=df.head(5000)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tok_messages = df['texto'].apply(lambda x: tokenizer(x)["input_ids"])

tok_messages = tok_messages.apply(lambda x: x[:300] + [0] * (300 - len(x)) if len(x) < 300 else x[:300])

df_tokenized = pd.DataFrame({'vector': tok_messages, 'name': df['n']})

df_tokenized.head()


/usr/local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


,vector,name
0,"[9126, 826, 314, 8618, 257, 1310, 2866, 351, 2...",13
1,"[2990, 389, 20140, 13666, 996, 355, 389, 477, ...",14
2,"[37, 5702, 453, 28336, 257, 16028, 318, 517, 6...",9
3,"[40, 1183, 423, 284, 48342, 616, 4088, 422, 40...",21
4,"[28042, 345, 460, 5368, 284, 2822, 477, 649, 4...",10


## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [19]:
db = lancedb.connect("./messages.lancedb")

db.create_table("tabla", df_tokenized)

db["tabla"].head()

pyarrow.Table
vector: fixed_size_list<item: float>[300]
  child 0, item: float
name: int64
----
vector: [[[9126,826,314,8618,257,...,0,0,0,0,0],[2990,389,20140,13666,996,...,0,0,0,0,0],[37,5702,453,28336,257,...,0,0,0,0,0],[40,1183,423,284,48342,...,0,0,0,0,0],[28042,345,460,5368,284,...,0,0,0,0,0]]]
name: [[13,14,9,21,10]]

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [36]:
result_df = (
    db["tabla"].search()
    
    # Filtra los resultados para incluir solo las filas donde la columna 'n' es mayor que 3
    .where("name > 3")
    
    # Selecciona únicamente las columnas 'n' y 'vector' de los resultados filtrados
    .select(["name", "vector"])
    
    # Limita el número de resultados a 11 filas
    .limit(11)
    
    # Convierte los resultados de la consulta a un DataFrame de pandas
    .to_pandas()
)

# Imprime el DataFrame resultante
print(result_df)


    name                                             vector
0     13  [9126.0, 826.0, 314.0, 8618.0, 257.0, 1310.0, ...
1     14  [2990.0, 389.0, 20140.0, 13666.0, 996.0, 355.0...
2      9  [37.0, 5702.0, 453.0, 28336.0, 257.0, 16028.0,...
3     21  [40.0, 1183.0, 423.0, 284.0, 48342.0, 616.0, 4...
4     10  [28042.0, 345.0, 460.0, 5368.0, 284.0, 2822.0,...
5     12  [15784.0, 329.0, 257.0, 2524.0, 326.0, 318.0, ...
6     24  [2949.0, 14148.0, 4001.0, 475.0, 345.0, 761.0,...
7      4  [2990.0, 389.0, 16001.0, 19943.0, 13.0, 0.0, 0...
8     15  [1858.0, 3729.0, 389.0, 517.0, 6792.0, 5672.0,...
9     15  [1639.0, 3551.0, 880.0, 11.0, 290.0, 1944.0, 3...
10    13  [464.0, 1097.0, 4444.0, 510.0, 852.0, 2861.0, ...


- Query en SQL equivalente:
- Explicacion: 

In [40]:
#SELECT name, vector
#FROM tabla
#WHERE name > 3
#LIMIT 11;

'''Explicación:

SELECT name, vector
Esta parte de la consulta selecciona las columnas name y vector de la tabla tabla. Es equivalente a .select(["name", "vector"]) en el código Python.

FROM tabla
Especifica la tabla tabla desde la cual se van a seleccionar los datos.

WHERE name > 3
Esta cláusula filtra las filas de la tabla para incluir solo aquellas donde el valor de la columna name es mayor que 3. Es equivalente a .where("name > 3") en el código Python.

LIMIT 11
Limita el número de filas devueltas por la consulta a 11. Es equivalente a .limit(11) en el código Python.'''

'Explicación:\nSELECT name, vector\n\nEsta parte de la consulta selecciona las columnas name y vector de la tabla tabla. Es equivalente a .select(["name", "vector"]) en el código Python.\nFROM tabla\n\nEspecifica la tabla tabla desde la cual se van a seleccionar los datos.\nWHERE name > 3\n\nEsta cláusula filtra las filas de la tabla para incluir solo aquellas donde el valor de la columna name es mayor que 3. Es equivalente a .where("name > 3") en el código Python.\nLIMIT 11\n\nLimita el número de filas devueltas por la consulta a 11. Es equivalente a .limit(11) en el código Python.'

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [24]:
mensaje = 'Hey, do you want to grab coffee later and catch up?'

mensaje_tok = tokenizer(mensaje)["input_ids"]

print(len(mensaje_tok))

if len(mensaje_tok) < 300:
    mi_mensaje_tokenized = mensaje_tok + [0] * (300 - len(mensaje_tok))
else:
    mi_mensaje_tokenized = mensaje_tok[:300]

df_q = (db["tabla"].search(mi_mensaje_tokenized)
        .metric("L2") 
        .select(["vector"])
        .limit(1)
        .to_pandas())

query_tok = df_q['vector'].values[0]

mensaje_parecido =  tokenizer.decode(query_tok, skip_special_tokens=True)
print(mensaje_parecido)

13
what are you thinking for the tail lights and that area?!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
